In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import requests
import time
import asyncio
import os

chrome_options = Options()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(chrome_options=chrome_options)

DOWNLOAD_FOLDER = r"D:\pyCharm\NoRoof\Download"

front_url = "https://www.ruten.com.tw/user/"

def check_folder(folder_path):
    if not folder_path.exists():
        Path.mkdir(folder_path, parents=True)

def go_to_directory(directory, *args):
    directory = Path(directory).joinpath(*args)
    check_folder(directory)
    os.chdir(directory)
    return directory


In [2]:

def download_img(path, ls): 
    for ele in ls:
        download_url = ele[0]
        file_name = ele[1]
        ir = requests.get(download_url)
        time.sleep(0.3)
        if ir.status_code == 200:
            open(f'{path}\\{file_name}', 'wb').write(ir.content)
            print(f"download '{path}\\{file_name}'")


In [3]:
# get all classification
driver.get("https://www.ruten.com.tw/user/index00.php?s=f2869397")
soup = BeautifulSoup(driver.page_source, "html.parser")
div = soup.select("div.sidebar-main.category-listing a")
# list of tuple [(link_url, title), ...]
classification_ls = [(front_url + u["href"], u.text) for u in soup.select("div.sidebar-main.category-listing a")[1:]]

In [4]:
# go to each of page
for classify in classification_ls:
    go_to_directory(DOWNLOAD_FOLDER, classify[1])
    # get all merchandise page
    driver.get(classify[0])
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # list of tuple [(link_url, title), ...]
    merchandise_ls = [(ele["href"], ele.text) for ele in soup.select("h3.item-name.isDefault-name a")]
    for merchandise in merchandise_ls:
        directory = go_to_directory(DOWNLOAD_FOLDER, classify[1], merchandise[1].rstrip())
        driver.get(merchandise[0])
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # list of tuple [(link_url, file_name), ...]
        side_img_ls = [(ele["src"], ele["src"].split("/")[-1]) for ele in soup.select("div.item-gallery img")]
        download_img(directory, side_img_ls)
        # list of tuple [(link_url, file_name), ...]
        below_img_ls = [(ele["src"], ele["src"].split("/")[-1]) for ele in soup.select("div.rt-tab-pane img")[1:-1]]
        download_img(directory, below_img_ls)